In [1]:
# billing https://platform.openai.com/settings/organization/billing/overview
# instalação do ffmpeg no windows: https://www.youtube.com/watch?v=JR36oH35Fgg
# https://ffmpeg.org/ffmpeg.html
# https://ffmpeg.org/download.html
# https://www.gyan.dev/ffmpeg/builds/ffmpeg-git-essentials.7z
# no Jupiter Notebook ffmpeg -i apresentacao.mp3 teste.avi

import gradio as gr
import whisper
from openai import OpenAI
import openai

import os
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env')

openai_api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai_api_key,
)

C:\Users\otimi\anaconda3\envs\python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# pip install whisper
# pip install gradio
# pip install openai-whisper
# pip install playsound
# pip install pygame
# pip install pydub

In [3]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("apresentacao.mp3", fp16=False)
print(result["text"])

# https://www.youtube.com/watch?v=UAdX0cGuC28
# Transcribe Audio Files with OpenAI Whisper

with open("transcription.txt","w") as f:
	f.write(result["text"])


 Curso completo de Langtian, com várias ferramentas úteis para seus projetos.


In [4]:
import os
from playsound import playsound

filename = "apresentacao.mp3"
filename = os.path.abspath(filename)
playsound(filename)

In [5]:
import pygame
import time

# Initialize pygame mixer
pygame.mixer.init()

# Load and play the sound
filename = "C:\\Users\\otimi\\Udemy\\apresentacao.mp3"
filename = "apresentacao.mp3"
pygame.mixer.music.load(filename)
pygame.mixer.music.play()

# Wait for the audio to finish
while pygame.mixer.music.get_busy():
    time.sleep(1)

# Stop the audio playback
pygame.mixer.music.stop()

# Clean up: Quit the pygame mixer and remove the temporary audio file
pygame.mixer.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
from pydub import AudioSegment
from playsound import playsound

# Convert MP3 to WAV
filename = "apresentacao.mp3"
filename = os.path.abspath(filename)
sound = AudioSegment.from_mp3(filename)
wav_filename = filename.replace('.mp3', '.wav')
sound.export(wav_filename, format="wav")

# Play the WAV file. Erro
playsound(filename) # ao executar em seguida dá erro....


In [7]:
import os

# Especifica o caminho do diretório
directory_path = './'

# Lista todos os arquivos e diretórios
files_and_directories = os.listdir(directory_path)

# Filtra apenas os arquivos
files = [f for f in files_and_directories if os.path.isfile(os.path.join(directory_path, f))]
mp3_files = [f for f in files_and_directories 
             if os.path.isfile(os.path.join(directory_path, f)) and f.lower().endswith('.mp3')]

print("Arquivos no diretório:", mp3_files)


Arquivos no diretório: ['apresentacao.mp3']


In [8]:
starting_prompt = """Você é um assistente.
Você pode discutir com o usuário.
Você receberá instruções começando com [Instruction] ou entrada do usuário começando com [User]. Siga as instruções.
"""

prompts = {
'START': '[Instrução] Escreva WRITE_EMAIL se o usuário quiser escrever um e-mail, "QUESTION" se o usuário tiver uma pergunta precisa, "OTHER" em qualquer outro caso. Escreva apenas uma palavra.',
'QUESTION': '[Instrução] Se você puder responder à pergunta, escreva "ANSWER", se precisar de mais informações, escreva MORE, se não puder responder, escreva "OTHER". Escreva apenas uma palavra.',
'ANSWER': '[Instrução] Responda à pergunta do usuário',
'MORE': '[Instrução] Peça mais informações ao usuário, conforme especificado nas instruções anteriores',
'OTHER': '[Instrução] Dê uma resposta educada ou saudações se o usuário estiver conversando educadamente. Caso contrário, responda ao usuário que você não pode responder à pergunta ou realizar a ação',
}

class Discussion:
    """
    A class representing a discussion with a voice assistant.

    Attributes:
        state (str): The current state of the discussion.
        messages_history (list): A list of dictionaries representing the history of messages in the discussion.
        client: An instance of the OpenAI client.
        stt_model: The speech-to-text model used for transcribing audio.

    Methods:
        generate_answer: Generates an answer based on the given messages.
        reset: Resets the discussion to the initial state.
        do_action: Performs the specified action.
        transcribe: Transcribes the given audio file.
        discuss_from_audio: Starts a discussion based on the transcribed audio file.
        discuss: Continues the discussion based on the given input.
    """

    def __init__(self, state='START', messages_history=None) -> None:
        if messages_history is None:
            messages_history = [{'role': 'user', 'content': f'{starting_prompt}'}]
        self.state = state
        self.messages_history = messages_history
        self.client = OpenAI()
        self.stt_model = whisper.load_model("base")

    def generate_answer(self, messages):
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        return response.choices[0].message.content

    def reset(self, start_state='START'):
        self.messages_history = [{'role': 'user', 'content': f'{starting_prompt}'}]
        self.state = start_state
        self.previous_state = None

    def reset_to_previous_state(self):
        self.state = self.previous_state
        self.previous_state = None

    def to_state(self, state):
        self.previous_state = self.state
        self.state = state

    def do_action(self, action):
        """
        Performs the specified action.

        Args:
            action (str): The action to perform.
        """
        print(f'DEBUG perform action={action}')
        pass

    def transcribe(self, file):
        transcription = self.stt_model.transcribe(file)
        return transcription['text']

    def discuss_from_audio(self, file):
        if file:
            # Transcribe the audio file and use the input to start the discussion
            return self.discuss(f'[User] {self.transcribe(file)}')
        # Empty output if there is no file
        return ''

    def discuss(self, input=None):
        if input is not None:
            self.messages_history.append({"role": "user", "content": input})

        # Generate a completion
        completion = self.generate_answer(
            self.messages_history +
            [{"role": "user", "content": prompts[self.state]}]
        )

        # Is the completion an action?
        actions = ["action1", "action2"]
        if completion.split("|")[0].strip() in actions:
            action = completion.split("|")[0].strip()
            self.to_state(action)
            self.do_action(completion)
            # Continue discussion
            return self.discuss()
        # Is the completion a new state?
        elif completion in prompts:
            self.to_state(completion)
            # Continue discussion
            return self.discuss()
        # Is the completion an output for the user?
        else:
            self.messages_history.append({"role": "assistant", "content": completion})
            if self.state != 'MORE':
                # Get back to start
                self.reset()
            else:
                # Get back to previous state
                self.reset_to_previous_state()
            return completion


# Ensure the class is defined before creating an instance
discussion = Discussion()

import gradio as gr

gr.Interface(
    theme=gr.themes.Soft(),
    fn=discussion.discuss_from_audio,
    live=True,
    inputs=gr.Audio(sources="microphone", type="filepath"),
    outputs="text",
    title="Transcrição de Áudio",
    description="Grave um áudio pelo microfone e obtenha a transcrição."
).launch()



* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\Users\otimi\anaconda3\envs\python\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 406, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\otimi\anaconda3\envs\python\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\otimi\anaconda3\envs\python\Lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "C:\Users\otimi\anaconda3\envs\python\Lib\site-packages\starlette\applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "C:\Users\otimi\anaconda3\envs\python\Lib\site-packages\starlette\middleware\errors.py", line 187, in __call__
    raise exc
  Fil

In [ ]:
# To use command line instead of Gradio, remove above code and use this instead:  Qual  acapital do Brasil ?
#while True:
message = input('User: ')
print(f'Assistant: {discussion.discuss(message)}')

In [9]:
import gradio as gr

# Função que retorna o caminho do arquivo de áudio
def play_audio():
    return "./apresentacao.mp3"

# Criação da interface Gradio
interface = gr.Interface(
    fn=play_audio,
    inputs=[],
    outputs=gr.Audio(type="filepath", label="Tocar Áudio"),
    title="Tocar Áudio MP3",
    description="Clique no botão Generate para tocar o áudio."
)

# Inicia a interface
interface.launch()



* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [10]:
import gradio as gr
import whisper

# Carregar o modelo de transcrição do Whisper
model = whisper.load_model("base")

# Função que retorna o caminho do arquivo de áudio e sua transcrição
def play_and_transcribe_audio():
    audio_path = "./apresentacao.mp3"
    transcription = model.transcribe(audio_path)
    return audio_path, transcription["text"]

# Criação da interface Gradio
interface = gr.Interface(
    fn=play_and_transcribe_audio,
    inputs=[],
    outputs=[gr.Audio(type="filepath", label="Tocar Áudio"), gr.Textbox(label="Transcrição")],
    title="Tocar Áudio MP3 e Transcrição",
    description="Clique no botão Generate para tocar o áudio e ver a transcrição."
)

# Inicia a interface
interface.launch()


C:\Users\otimi\anaconda3\envs\python\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


C:\Users\otimi\anaconda3\envs\python\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
